# Grandmaster's Path: Decoding Difficulty and Winning Strategies in Wikispeedia

In this notebook, we present the most important things we noticed when analysing the dataset. We then use them to implement interesting additions to the Wikispeedia game, such as inherent difficulty of missions, ingame analysis of played games and an Elo system for players. We propose a way of calculating in-game scores and Elo based on the already existing chess methods.

Our main goal was to gamify Wikispeedia, which makes the game more strategic and motivates players to play thanks to the Elo system and given feedback. Using results from this notebook, the game can be modified in a way where players are given hints at the right time and are given missions of appropriate difficulty.

In [5]:
import numpy as np
import pandas as pd
import os
import sys

from matplotlib import pyplot as plt
from urllib.parse import unquote
import seaborn as sns
import math 
import scipy.stats as stats
import statsmodels.api as sm
import random
import sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

from utils.data_processing import *
from utils.graph_processing import *
from utils.ingame_processing import *
from models.logistic_regression import LogisticRegression

## Main observations from the dataset

In this section, we present our main findings during the preprocessing of the dataset.

In [10]:
articles_df = load_article_df("./data/paths-and-graph/articles.tsv")
links_df = load_links_df("./data/paths-and-graph/links.tsv")
adj_matrix = construct_adjecency_matrix(links_df, articles_df['article_name'].tolist())

Loaded 4604 articles in df of shape (4604, 1)
Loaded 119882 links in df of shape (119882, 2)
After adding missing links, there are 119888 links in df


### Articles and links

## Inherent difficulty

## In-game tracking

## Elo system